In [1]:
import networkx as nx
import random
import copy
import time
import matplotlib.pyplot as plt
import sys
from collections import defaultdict
%matplotlib inline

def progressBar(value, endvalue, bar_length=20):

        percent = float(value) / endvalue
        arrow = '-' * int(round(percent * bar_length)-1) + '>'
        spaces = ' ' * (bar_length - len(arrow))

        sys.stdout.write("\rCalculating: [{0}] {1}%".format(arrow + spaces, int(round(percent * 100))))
        sys.stdout.flush()

        
def kawai_draw(G, infected=[]) :
    colors = []
    for node in G :
        if node in infected :
            colors.append('red')
        else :
            colors.append('green')
    nx.draw(G, layout=nx.kamada_kawai_layout(G), node_color=colors, node_size = 30, alpha = 0.7)
    plt.show()
    
num_nodes = 2000
    
G = nx.scale_free_graph(num_nodes).to_undirected()

#G = nx.watts_strogatz_graph(2000, 10, 1)

In [2]:
#kawai_draw(G)

In [3]:
import networkx as nx
import copy
import time
start = time.time()

spl = nx.all_pairs_shortest_path_length(G)

def change_ds(spl_gen, num_nodes, max_radius):
    ''' Transforms 'spl_gen' into the accepted datastructure
        by the 'infected_average_by_separation_degree' method
    '''
    result = []
    for n in range(num_nodes):
        progressBar(n, num_nodes, 50)
        di_tmp = {}
        for entry in spl_gen:  
            for radius in range(1, max_radius + 1):
                li_tmp = []
                for node, dist in entry[1].items():
                    if dist == radius:
                        li_tmp.append(node)
                if li_tmp != []:
                    di_tmp[radius - 1] = li_tmp
            result.append(copy.copy(di_tmp))
            
    return result

separation_degrees = change_ds(spl, num_nodes, 6)
print('\n', time.time() - start)

Calculating: [------------------------------------------------->] 100%
 19.12777805328369


In [4]:
def infected_percentage(nodes, infected) :
    count = 0
    for node in nodes :
        if node in infected :
            count += 1
    
    return count / len(nodes)

def infected_average_by_separation_degree(G, separation_degrees, infected) :
    node_count = {}
    percentage_sum = {}
    
    for node in range(0, len(separation_degrees)) :
        if node in infected :
            for degree in separation_degrees[node] :
                if degree not in percentage_sum.keys() :
                    percentage_sum[degree] = 0
                    node_count[degree] = 0
                percentage_sum[degree] += infected_percentage(separation_degrees[node][degree], infected)
                node_count[degree] += 1
        
    average_by_separation_degree = {}
    
    for separation_degree in percentage_sum :
        average_by_separation_degree[separation_degree] = percentage_sum[separation_degree] / node_count[separation_degree]
            
    return average_by_separation_degree


In [5]:
import networkx as nx
import ndlib.models.ModelConfig as mc
import ndlib.models.epidemics.SIModel as si
import ndlib.models.opinions.VoterModel as vt

# Model selection
#model = si.SIModel(G)
model = vt.VoterModel(G)

# Model Configuration
cfg = mc.Configuration()
cfg.add_model_parameter("percentage_infected", 0.5) # initially infected  %
model.set_initial_status(cfg)

# Simulation execution
iterations = model.iteration_bunch(1000)

In [6]:
infected = []
infected_count = []
avg_traits = []
for it in range(0, len(iterations)) :
    for node, i in iterations[it]['status'].items() :
        if i == 1:
            infected.append(node)
            
    if it % 50 == 0 :
        percentages = infected_average_by_separation_degree(G, separation_degrees, infected)
        
        print('Iteration', it+1)
        print('Infected count', len(infected))
        print('Separations:')
        print(percentages)

        avg_traits.append(percentages) # needed for propensity
        infected_count.append(len(infected)) # needed for the random graph generation

Iteration 1
Infected count 1000
Separations:
{0: 0.6348341739585425, 1: 0.5196967271590791, 2: 0.5028615107899748, 3: 0.4984038678011384, 4: 0.49020139812111263, 5: 0.33358140665809605}
Iteration 51
Infected count 1028
Separations:
{0: 0.6454819089386262, 1: 0.5290174652505227, 2: 0.5098876769763812, 3: 0.5059490020226932, 4: 0.4950188929068754, 5: 0.3336182790743591}
Iteration 101
Infected count 1064
Separations:
{0: 0.6499667814033631, 1: 0.5340545730169435, 2: 0.5185157217269988, 3: 0.5110322281640247, 4: 0.4966918380291558, 5: 0.33434740104634925}
Iteration 151
Infected count 1095
Separations:
{0: 0.6550624915471591, 1: 0.539591739421546, 2: 0.525536845156023, 3: 0.5150493636871908, 4: 0.4981298152047041, 5: 0.333663501504946}
Iteration 201
Infected count 1127
Separations:
{0: 0.6630416154804633, 1: 0.5476866792548528, 2: 0.5336268664054631, 3: 0.5210098837331442, 4: 0.5006773241889353, 5: 0.3346096505251183}
Iteration 251
Infected count 1157
Separations:
{0: 0.668874365407183, 1: 

In [7]:
''' Start Random distribution of infecteds section '''
import random
from collections import defaultdict
from copy import deepcopy # only used for testing

def simulate_random_infecteds(G_rand, num_nodes, radius_split_rand, infected_count):
    
    '''
    Randomly picks the same number of infecteds as in the 
    non-random model (for a each iteration of the spreading)
    Returns a list of @infected_average_by_radius lists
    '''
    print(infected_count)
    random_infected = []
    random_not_infected = list(range(0, num_nodes))
    prev_num_infects = 0
    new_num_infects = 0
    avg_trait_sharing_by_radius = []
    for num_infects in infected_count:
        print("num_infects: ", num_infects)
        new_num_infects = num_infects - prev_num_infects
        print("new infecteds: ", new_num_infects)
        # prev_random_infected = deepcopy(random_infected) # for testing purposes

        for _ in range(new_num_infects):
            if len(random_not_infected) == 0 :
                break
            new_inf = random.choice(random_not_infected)
            random_infected.append(new_inf)
            random_not_infected.remove(new_inf)
            
        if len(random_not_infected) == 0 :
            break   
            
        prev_num_infects = num_infects
        percentage = infected_average_by_separation_degree(G_rand, separation_degrees, random_infected)
        avg_trait_sharing_by_radius.append(percentage)
        print(percentage)    
        
        # print("¨¨¨¨¨¨¨¨¨Infected list: ", len(random_infected), " nodes\n", random_infected)
        # print("¨¨¨¨¨¨¨¨¨NOT Infected list:", len(random_not_infected), " nodes\n", random_not_infected) 
        # print_list_differences(prev_random_infected, random_infected)
    return (avg_trait_sharing_by_radius, random_infected)

avg_random_trait_sharing_by_radius, final_random_infected = simulate_random_infecteds(G, num_nodes, separation_degrees, infected_count)

[1000, 1028, 1064, 1095, 1127, 1157, 1189, 1216, 1244, 1270, 1300, 1328, 1355, 1383, 1413, 1441, 1461, 1482, 1510, 1538]
num_infects:  1000
new infecteds:  1000
{0: 0.42549769497035234, 1: 0.5038538165378464, 2: 0.49961323389293444, 3: 0.490099291697853, 4: 0.4877420654632084, 5: 0.7683192935866698}
num_infects:  1028
new infecteds:  28
{0: 0.42624923609232906, 1: 0.5188601598569273, 2: 0.5126813712274795, 3: 0.504253781569925, 4: 0.4925933571330284, 5: 0.7702027346050719}
num_infects:  1064
new infecteds:  36
{0: 0.43808280543019706, 1: 0.5322382444620035, 2: 0.5286417706229506, 3: 0.5237772037475108, 4: 0.5058363085543246, 5: 0.7747562284038845}
num_infects:  1095
new infecteds:  31
{0: 0.4635665267779598, 1: 0.551353491908208, 2: 0.5447911195564316, 3: 0.5386171454678716, 4: 0.5102560761671832, 5: 0.7752526010023412}
num_infects:  1127
new infecteds:  32
{0: 0.47228413409062, 1: 0.5639536753011406, 2: 0.5592811207920791, 3: 0.5577393808938722, 4: 0.5289600124697643, 5: 0.77875007781

In [16]:
def propensity_by_radius(iteration, radius, avg_traits, rand_avg_traits):
    return (avg_traits[iteration-1][radius-1] / rand_avg_traits[iteration-1][radius-1]) - 1

def all_propensities(iteration, max_radius, avg_traits, rand_avg_traits):
    all_propensities = []
    print(iteration)
    for r in range(1, max_radius + 1):
        print(r)
        
        all_propensities.append(propensity_by_radius(iteration, r, avg_traits, rand_avg_traits))
    return all_propensities

prop_one = propensity_by_radius(10, 1, avg_traits, avg_random_trait_sharing_by_radius)
prop_two = propensity_by_radius(10, 2, avg_traits, avg_random_trait_sharing_by_radius)
print(prop_one)

all_props = all_propensities(2, 6, avg_traits, avg_random_trait_sharing_by_radius)
print(all_props)

def normalized_propensities(max_radius, all_propensities):
    normalized_propensities = []
    for r in range(max_radius):
        normalized_propensities.append(all_propensities[r] / all_propensities[0])
    
    return normalized_propensities
        
print(normalized_propensities(6, all_props))

0.31965625625003824
2
1
2
3
4
5
6
[0.5143297730129177, 0.019576190618289413, -0.005449182295056976, 0.0033618398408246097, 0.004924012349585949, -0.566843554190411]
[1.0, 0.03806155436737228, -0.010594724593009544, 0.006536350834078926, 0.009573648285498493, -1.1021013830676576]


In [9]:
#Do not delete, create list for each degree and add to report
avg = 0
for node in separation_degrees :
    if len(node) > 5 :
        avg += len(node[5])

print(avg / len(separation_degrees))

26.463


In [10]:
print(2 * G.number_of_edges() / G.number_of_nodes())

4.247


In [11]:
'''import pickle

with open('2000_nodes_scale_free.graph', 'wb') as f:  # Python 3: open(..., 'wb')
    pickle.dump(G, f)'''

"import pickle\n\nwith open('2000_nodes_scale_free.graph', 'wb') as f:  # Python 3: open(..., 'wb')\n    pickle.dump(G, f)"

In [12]:
'''import pickle

with open('2000_nodes_scale_free.graph', 'rb') as f:  # Python 3: open(..., 'rb')
    Gtest = pickle.load(f)

print(2 * Gtest.number_of_edges() / Gtest.number_of_nodes())'''

"import pickle\n\nwith open('2000_nodes_scale_free.graph', 'rb') as f:  # Python 3: open(..., 'rb')\n    Gtest = pickle.load(f)\n\nprint(2 * Gtest.number_of_edges() / Gtest.number_of_nodes())"